## BLEU: BiLingual Evaluation Understudy

### 1. APIs

In [1]:
import nltk
import nltk.translate.bleu_score as bleu


import math
import numpy
import os

try:
  nltk.data.find('tokenizers/punkt')
except LookupError:
  nltk.download('punkt')

### 2. Jeu de données
Tableau de mots : les phrases candidates et de référence sont segmentées en mots

In [2]:
hyp = str('she read the book because she was interested in world history').split()
ref_a = str('she read the book because she was interested in world history').split()
ref_b = str('she was interested in world history because she read the book').split()

### 3. Score d'une phrase
Compare 1 hypothèse (candidate) avec une ou plusieurs références et retourne le plus haut score

In [3]:
score_ref_a = bleu.sentence_bleu([ref_a], hyp)
print("Hyp and ref_a are the same: {}".format(score_ref_a))
score_ref_b = bleu.sentence_bleu([ref_b], hyp)
print("Hyp and ref_b are different: {}".format(score_ref_b))
score_ref_ab = bleu.sentence_bleu([ref_a, ref_b], hyp)
print("Hyp vs multiple refs: {}".format(score_ref_ab))

Hyp and ref_a are the same: 1.0
Hyp and ref_b are different: 0.7400828044922853
Hyp vs multiple refs: 1.0


### 4. Score du corpus

Ce score est différent de la moyenne des scores BLEU de chaque phrase, il calcule le score en sommant les numérateurs et les dénominateurs pour chaque paire hypothèse-référence(s) avant la division


In [4]:
score_ref_a = bleu.corpus_bleu([[ref_a]], [hyp])
print("1 document with 1 reference sentence: {}".format(score_ref_a))
score_ref_a = bleu.corpus_bleu([[ref_a, ref_b]], [hyp])
print("1 document with 2 reference sentences: {}".format(score_ref_a))
score_ref_a = bleu.corpus_bleu([[ref_a], [ref_b]], [hyp, hyp])
print("2 documents with 1 reference sentence each: {}".format(score_ref_a))

1 document with 1 reference sentence: 1.0
1 document with 2 reference sentences: 1.0
2 documents with 1 reference sentence each: 0.8778107713916036


### 5. BLEU-$n$
Dans BLEU-$n$, les scores $n$-gram peuvent être obtenus dans les calculs de **phrase** et de **corpus** et ils sont indiqués par le paramètre **weights**.

* *weights* : longueur 4, où chaque index contient un poids correspondant à son $n$-gramme respectif.
* $n$-gramme avec $n \in \{1, 2, 3, 4\}$
* $\textit{weights}=(W_{N=1}, W_{N=2}, W_{N=3}, W_{N=4})$


In [5]:
score_1gram = bleu.sentence_bleu([ref_b], hyp, weights=(1,0,0,0))
score_2gram = bleu.sentence_bleu([ref_b], hyp, weights=(0,1,0,0))
score_3gram = bleu.sentence_bleu([ref_b], hyp, weights=(0,0,1,0))
score_4gram = bleu.sentence_bleu([ref_b], hyp, weights=(0,0,0,1))
print("N-grams: 1-{}, 2-{}, 3-{}, 4-{}".format(score_1gram, score_2gram, score_3gram, score_4gram))

N-grams: 1-1.0, 2-0.9, 3-0.6666666666666666, 4-0.5


N-grammes cumulés : par défaut, le score est calculé en considérant tous les $N$-grammes de manière égale dans une moyenne géométrique

In [9]:
score_ngram1 = bleu.sentence_bleu([ref_b], hyp)
score_ngram = bleu.sentence_bleu([ref_b], hyp, weights=(0.25,0.25,0.25,0.25))
score_ngram_geo = (11/11*9/10*6/9*4/8)**0.25
print("N-grams: {} = {} = {}".format(score_ngram1, score_ngram, score_ngram_geo))

N-grams: 0.7400828044922853 = 0.7400828044922853 = 0.7400828044922853


In [10]:
list_of_references = [[ref_a, ref_b]]
hypotheses = [hyp]
bleu.corpus_bleu(list_of_references, hypotheses) 

1.0

In [11]:
hyp_len = len(hyp)
closest_ref_len =  bleu.closest_ref_length(list_of_references, hyp_len)
bleu.brevity_penalty(closest_ref_len, hyp_len)

1